In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import statsmodels.api as sm
import pandas as pd
import plotly.express as px
from scipy.signal import savgol_filter
from statsmodels.stats.outliers_influence import variance_inflation_factor
from linearmodels.panel import PanelOLS




In [3]:
bank_names_mapping = mapping_dict = {
    'banco': 'banc sabadell',
    'my': 'myinvestor',
    'evo': 'evobanco',
    'caixa': 'caixabank',
    'abanca': 'abanca',
    'banc sabadell': 'banc sabadell',
    'bancosabadell': 'banc sabadell',
    'bancosantander': 'santander',
    'bancsabadell': 'banc sabadell',
    'sabadell': 'banc sabadell',
    'bank': 'bank',
    'bankinter': 'bankinter',
    'bbva': 'bbva',
    'caixabank': 'caixabank',
    'caixa_bank': 'caixabank',
    'date': 'date',
    'evobanco': 'evobanco',
    'evo_banco': 'evobanco',
    'imagin': 'imagin',
    'ing': 'ing',
    'ing-bank': 'ing',
    'ing_bank': 'ing',
    'kutxabank': 'kutxabank',
    'liberbank': 'liberbank',
    'myinvestor': 'myinvestor',
    'n26': 'n26',
    'n26.com': 'n26',
    None: None,  # handling nan (not a number)
    'openbank': 'openbank',
    'orangebank': 'orangebank',
    'pibank': 'pibank',
    'renaultbank': 'renaultbank',
    'revolut': 'revolut',
    'santander': 'santander',
    'value': 'value'
}

In [4]:
data = pd.read_csv('data/regression_clean_data_final_1.csv')
data.head()

,bank,date,web_traffic,cross_visitation,search_interest,rank,incentive,apr,mentions,app_installs,ppc_spend
0,abanca,2021-12-01,15202.983835,0.077994,7.75,4.0,150.0,0.0,1.0,43605.699637,1506.110026
1,abanca,2022-01-01,42183.913207,0.062500,7.80,4.0,150.0,0.0,1.0,43605.699637,1506.110026
2,abanca,2022-02-01,34498.153115,0.052288,7.25,4.0,150.0,0.0,2.0,48647.434439,32750.639361
3,abanca,2022-03-01,34546.319021,0.061503,6.75,4.0,150.0,0.0,2.0,26687.227413,15754.216964
4,abanca,2022-04-01,40508.288420,0.050222,6.75,4.0,300.0,0.0,2.0,4254.184995,6542.970638


In [8]:
# view avg ppc grouped by bank in plotly bar graph 

grouped_data = data.groupby('bank')['ppc_spend'].mean().reset_index()
# sort the data
grouped_data = grouped_data.sort_values('ppc_spend', ascending=False)

fig = px.bar(grouped_data, x='bank', y='ppc_spend')
fig.show()

In [9]:
# view avg ppc grouped by bank in plotly bar graph 

grouped_data = data.groupby('bank')[['ppc_spend', 'web_traffic']].mean().reset_index()
# sort the data
grouped_data = grouped_data.sort_values('ppc_spend', ascending=False)

#plot both web traffic and ppc spend in a grouped bar 
fig = go.Figure(data=[
    go.Bar(name='ppc_spend', x=grouped_data['bank'], y=grouped_data['ppc_spend']),
    go.Bar(name='web_traffic', x=grouped_data['bank'], y=grouped_data['web_traffic'])
])


fig.show()

In [11]:
# view avg ppc grouped by bank in plotly bar graph 

grouped_data = data.groupby('bank')[['app_installs', 'web_traffic']].mean().reset_index()
# sort the data
grouped_data = grouped_data.sort_values('app_installs', ascending=False)

#plot both web traffic and ppc spend in a grouped bar 
fig = go.Figure(data=[
    go.Bar(name='app_installs', x=grouped_data['bank'], y=grouped_data['app_installs']),
    go.Bar(name='web_traffic', x=grouped_data['bank'], y=grouped_data['web_traffic'])
])


fig.show()

In [14]:
bank_app_data = {
    "bbva": {
        "downloads": "10M+",
        "rating": 4.6
    },
    "santander": {
        "downloads": "10M+",
        "rating": 4.7
    },
    "revolut": {
        "downloads": "10M+",
        "rating": 4.7
    },
    "banc sabadell": {
        "downloads": "5M+",
        "rating": 4.6
    },
    "abanca": {
        "downloads": "1M+",
        "rating": 4.4
    },
    "ing": {
        "downloads": "1M+",
        "rating": 4.8
    },
    "openbank": {
        "downloads": "1M+",
        "rating": 3.9
    },
    "bankinter": {
        "downloads": "1M+",
        "rating": 3.9
    },
    "evobanco": {
        "downloads": "1M+",
        "rating": 4.2
    },
    "n26": {
        "downloads": "1M+",
        "rating": 3.6
    },
    "myinvestor": {
        "downloads": "1M+",
        "rating": 2.2
    }
}

bank_app_data = pd.DataFrame(bank_app_data).T.reset_index()

bank_app_data.columns = ['bank', 'downloads', 'rating']

bank_app_data





,bank,downloads,rating
0,bbva,10M+,4.6
1,santander,10M+,4.7
2,revolut,10M+,4.7
3,banc sabadell,5M+,4.6
4,abanca,1M+,4.4
5,ing,1M+,4.8
6,openbank,1M+,3.9
7,bankinter,1M+,3.9
8,evobanco,1M+,4.2
9,n26,1M+,3.6


In [15]:

grouped_data = data.groupby('bank')[['app_installs', 'web_traffic']].mean().reset_index()
# sort the data
grouped_data = grouped_data.sort_values('app_installs', ascending=False)

# attach ratings to grouped data

grouped_data = pd.merge(grouped_data, bank_app_data, on='bank', how='left')

grouped_data

,bank,app_installs,web_traffic,downloads,rating
0,bbva,98488.970652,39683.313890,10M+,4.6
1,santander,88544.563646,39051.034753,10M+,4.7
2,revolut,65119.021615,5878.371419,10M+,4.7
3,banc sabadell,44285.344383,29385.767853,5M+,4.6
4,abanca,41747.782541,54317.650012,1M+,4.4
5,ing,41185.188848,28067.720768,1M+,4.8
6,openbank,21218.746465,16975.674789,1M+,3.9
7,bankinter,14945.346685,11021.738528,1M+,3.9
8,evobanco,12266.060562,6785.136961,1M+,4.2
9,n26,9287.109492,8124.031548,1M+,3.6


In [25]:
# plot web traffic, app installs and ratings in a grouped bar graph with ratings on a secondary y axis


fig = go.Figure()

# Bar trace for app installs
fig.add_trace(go.Bar(
    name='app_installs',
    x=grouped_data['bank'],
    y=grouped_data['app_installs'],
    marker_color='#62B5E5'
))

# Bar trace for web traffic
fig.add_trace(go.Bar(
    name='web_traffic',
    x=grouped_data['bank'],
    y=grouped_data['web_traffic'],
    marker_color='#00A3E0'
))

# Scatter trace for ratings
fig.add_trace(go.Scatter(
    name='rating',
    x=grouped_data['bank'],
    y=grouped_data['rating'],
    yaxis='y2',
    mode='markers',
    marker=dict(color='#012169')
))

# Update layout to add a secondary y-axis
fig.update_layout(
    yaxis2=dict(
        title='Rating',
        overlaying='y',
        side='right'
    ),
    barmode='group'
)

# add title 
fig.update_layout(title='App installs, Web traffic and Ratings by Bank')

fig.show()